# Titanic Data Science Solutions

<h2> This notebook is a companion to the book <a href='https://www.amazon.com/Data-Science-Solutions-Startup-Workflow/dp/1520545312' target='blank'>Data Science Solutions.</a></h2>

The notebook walks us through a typical workflow for solving data science competitions at sites like Kaggle.

There are several excellent notebooks to study data science competition entries. However many will skip some of the explanation on how the solution is developed as these notebooks are developed by experts for experts. The objective of this notebook is to follow a step-by-step workflow, explaining each step and rationale for every decision we take during solution development.

<h2>Workflow stages</h2>

The competition solution workflow goes through seven stages described in the Data Science Solutions book.

<ol>
    <li>Question or problem definition.</li>
    <li>Acquire training and testing data.</li>
    <li>Wrangle, prepare, cleanse the data.</li>
    <li>Analyze, identify patterns, and explore the data.</li>
    <li>Model, predict and solve the problem.</li>
    <li>Visualize, report, and present the problem solving steps and final solution.</li>
    <li>Supply or submit the results.</li>
</ol>

The workflow indicates general sequence of how each stage may foolow the other. However there are use cases with exceptions.

<ul>
    <li>We may combine multiple workflow stages. We may analyze by visualizing data.</li>
    <li>Perform a stage earlier than indicated. We may analyze data before and after wrangling.</li>
    <li>Perform a stage multiple times in our workflow. Visualize stage may be used multiple times.</li>
    <li>Drop a stage altogether. We may not need supply stage to productize or service enable our dataset for a competition.</li>
</ul>

<h2>Question and problem definition</h2>

Competition sites like Kaggle define the problem to solve or questions to ask while providing the datasets for training your data science model and testing the model results against a test dataset. The question or problem definition for Titanic Survival competition is <a href='https://www.kaggle.com/c/titanic' target='blank'>described here at Kaggle.</a>

>Knowing from a training set of samples listing passengers who survived or did not survivie the Titanic disaster, can our model determine based on a given test dataset not containing the survival information, if these passengers in the test dataset survived or not.

We may also want to develop some early understanding about the domain of our problem. This is described on the <a href='https://www.kaggle.com/c/titanic' target='blank'>Kaggle competition description page here.</a> Here are the highlights to note.

<ul>
    <li>On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. Translated 32% survival rate.</li>
    <li>One of the reasons that the shipwreck led to such loss of life was that there were not enough life boats for the passengers and crew.</li>
    <li>Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.</li>
</ul>

<h3>Workflow goals</h3>

The data science solutions workflow solves for seven major goals.

<b>Classifying.</b> We may want to classify or categorize our samples. We may also want to understand the implications or correlation of different classes with our solution goal.

<b>Correlating.</b> One can approach the problem based on available features within the training dataset. Which features within the dataset contribute significantly to our solution goal? Statisticaaly speaking is there a <a href='https://en.wikiversity.org/wiki/Correlation' target='blank'>correlation</a> among a feature and solution goal? As the feature values change does the solution state change as well, and vice-versa? This can be tested both for numerical and categorical features in the given dataset. We may also want to determine correlation among features other than survival for subsequent goals and workflow stages. Correlating certain features may help in creating, completing, or correcting features.

<b>Converting.</b> For modeling stage, one needs to prepare the data. Depending on the choice of model algorithm one may require all features to be converted to numerical equivalent values. So for instance converting text categorical values to numeric values.

<b>Completing.</b> Data preparation may also require us to estimate any missing values within a features. Model algorithms may work best when there are no missing values.

<b>Correcting.</b> We may also analyze the given training dataset for errors or possibly innacurate values within features and try to corrent these values or exclude the samples containing the errors. One way to do this is to detect any outliers among our samples or features. We may also completely discard a feature if it is not contributing to the analysis or may significantly skew the results.

<b>Creating.</b> Can we create new features based on an existing feature or a set of features, such that the new feature follows the correlation, conversion, completeness goals.

<b>Charting.</b> How to select the right visualization plots and charts depending on nature of the data and the solution golas.

### Refactor Release 2017-Jan-29

We are significantly refactoring the notebook based on (a) comments received by readers, (b) issues in porting notebook from Jupyter kernel (2.7) to Kaggle kernel(3.5), and (c) review of few more best practie kernels.

<b>User comments</b>
<ul>
    <li>Combine training and test data for certain operations like converting titles across dataset to numerical values.</li>
    <li>Correct observation - nearly 30% of the passengers had siblings and/or spouses aboard.</li>
    <li>Correctly interpreting logistic regression coefficients.</li>
</ul>

<b>Porting issues</b>
<ul>
    <li>Specify plot dimensions, bring legend into plot.</li>
</ul>

<b>Best practies</b>
<ul>
    <li>Performing feature correlation analysis early in the project.</li>
    <li>Using multiple plots instead of overlays for readability.</li>
</ul>

In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

### Acquire data

The Python Pandas packages helps us work with our datasets. We start by acquiring the training and testing datasets into Pandas DataFrames. We also combine these datasets to run certain operations on both datasets together.

In [3]:
train_df = pd.read_csv('./datasets/titanic/train.csv')
test_df = pd.read_csv('./datasets/titanic/test.csv')
combine = [train_df, test_df] # 2개의 DataFrame list

### Analyze by describing data

Pandas also helps describe the datasets answering following questions early in our project.

<b>Which features are available inthe dataset?</b>

Noting the feature names for directly manipulating or analyzing these. These feature names are described on the <a href='https://www.kaggle.com/c/titanic/data' target='blank'>Kaggle data page here.</a>

In [8]:
print(train_df.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


<b>Which features are categorical?</b>

These values classify the samples into sets of similar samples. Within categorical features are the values nominal, ordinal, ratio, or interval based? Among other things this helps us select the appropriate plots for visualiation.

<li>Categorical: Survived, Sex, and Embarked. Ordinal: Pclass.</li>

<b>Which features are numerical?</b>

Which features are numerical? These values change from sample to sample. Within numerical features are the values discrete, continuous, or timeseries based? Among other things this helps us select the appropriate plots for visualization.

<li>Continous: Age, Fare. Discrete: SibSp, Parch.</li>

In [9]:
# preview the data
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


<b>Which features are mixed data types?</b>

Numerical, alphanumeric data within same feature. These are candidates for correcting goal.

<li>Ticket is a mix of numeric and alphanumeric data typs. Cabin is alphanumeric.</li>

<b>Which features may contain errors or typos?</b>

This is harder to review for a large dataset, however reviewing a few samples from a smaller dataset may just tell us outright, which features may require correcting.

<li>Name feature may contain errors or typos as thes are several ways used to describe a name including titles, round bracktes, and quotes used for alternative or short names.</li>

In [10]:
train_df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


<b>Which features contain blank, null or empty values?</b>

These will require correcting.

<ul>
    <li>Cabin > Age > Embarked features contain a number of null values in that order for the training dataset.</li>
    <li>Cabin > Age are incomplete in case of test dataset.</li>
</ul>

<b>What are the data typse for various features?</b>

Helping us during converting goal.

<ul>
    <li>Seven features are integer or floats. Six in case of test dataset.</li>
    <li>Five features are strings(object).</li>
</ul>

In [11]:
train_df.info()
print('_' * 40)
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenger

<b>What is the distribution of numerical feature values across the samples?</b>

This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.

<ul>
    <li>Total samples are 891 or 40% of the actual number of passengers on board the Titanic(2,224).</li>
    <li>Survived is a categorical feature with 0 or 1 values.</li>
    <li>Around 38% samples survived representative of the actual survival rate at 32%.</li>
    <li>Most passengers (>75%) did not travel with parents or children.</li>
    <li>Nearly 30% of the passengers had siblings and/or spouse aboard.</li>
    <li>Fares varied significantly with few passengers (<1%) paying as high as \$512.</li>
    <li>Few elderly passengers (<1%) within age range 65-80.</li>
</ul>

In [22]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [17]:
# Review survived rate using 'percentils=[.61, .62]' knowing our problem
# description mentions 38$ survival rate.
train_df.describe(percentiles=[.61, .62])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
61%,543.900000,0.000000,3.000000,32.000000,0.000000,0.000000,23.225000
62%,552.800000,1.000000,3.000000,32.000000,0.000000,0.000000,24.150000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [19]:
# Review Parch distribution using 'percentiles=[.75, .8]'
train_df.describe(percentiles=[.75, .8])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
80%,713.000000,1.000000,3.000000,41.000000,1.000000,1.000000,39.687500
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [21]:
# Sibsp distribution '[.68, .69]'
train_df.describe(percentiles=[.68, .69])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
68%,606.200000,1.000000,3.000000,35.000000,0.000000,0.000000,26.307500
69%,615.100000,1.000000,3.000000,35.000000,1.000000,0.000000,26.550000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [23]:
# Age and Fare '[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99]'
train_df.describe(percentiles=[.1, .2, .3, .4, .5, .6, .7, .8, .9, .99])

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
10%,90.000000,0.000000,1.000000,14.000000,0.000000,0.000000,7.550000
20%,179.000000,0.000000,1.000000,19.000000,0.000000,0.000000,7.854200
30%,268.000000,0.000000,2.000000,22.000000,0.000000,0.000000,8.050000
40%,357.000000,0.000000,2.000000,25.000000,0.000000,0.000000,10.500000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
60%,535.000000,0.000000,3.000000,31.800000,0.000000,0.000000,21.679200


<b>What is the distribution of categorical features?</b>

<ul>
    <li>Names are unique across the dataset (count=unique=891)</li>
    <li>Sex variable as two possible values with 65% male (top=male, freq=877/count=891).</li>
    <li>Cabin values have several duplicates across samples. Alternatively several passengers shared a cabin.</li>
    <li>Embarked takes these possible values. S port used by most passengers (top=S)</li>
    <li>Ticket feature has high ratio (22%) of duplicate values (unique=681).</li>
</ul>

In [24]:
train_df.describe(include='O')

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644
